In [ ]:
# The core idea of agents is to use a language model to choose a 
# sequence of actions to take. In chains, a sequence of actions is
# hardcoded (in code). In agents, a language model is used as a
# reasoning engine to determine which actions to take and in which
# order.

In [ ]:
# Create a LLM
from getpass import getpass
import os
os.environ['OPENAI_API_KEY'] = getpass()

from langchain_community.chat_models import ChatOpenAI
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

In [ ]:
# Get the prompt to use - you can modify this!
from langchain import hub
prompt = hub.pull("hwchase17/self-ask-with-search")
# print(f'{prompt.format(agent_scratchpad = "AGENTSCRATCHPAD", input = "INPUT")}')

In [ ]:
# Define the tool to use. This will be a retreiver_tool.
#
# A retriever is an interface that returns documents given an unstructured query.
# It is more general than a vector store. A retriever does not need to be able to store
# documents, only to return (or retrieve) them. Vector stores can be used as the backbone
# of a retriever, but there are other types of retrievers as well.

# Connect to a web service
from langchain.retrievers import ArxivRetriever
from langchain.retrievers.web_research import WebResearchRetriever
axriv_retriever = ArxivRetriever(load_max_docs=2, sortBy='lastUpdatedDate', name='Intermediate Answer')
web_research_retreiver = WebResearchRetriever()

retreiver = web_research_retreiver

query = "recent flavivirus outbreak"
docs = retriever.get_relevant_documents(query=query)
for index, doc in enumerate(docs):
    print ("{}\t{}.".format(index, doc.metadata['Title']))
    print("\t{}".format(doc.metadata['Entry ID']))
    print("\t{}".format(doc.metadata['Published']))
    # print(doc.metadata)

In [ ]:
from langchain.tools.retriever import create_retriever_tool

# retriever – The retriever to use for the retrieval
# name – The name for the tool. This will be passed to the language model, 
#   so it should be unique and somewhat descriptive.
# description – The description for the tool. This will be passed to the language model,
#   so it should be descriptive.

retriever_tool = create_retriever_tool(
    retriever,
    "Intermediate Answer",
    "Search for information about outbreaks. For any questions about outbreaks, you must use this tool!",
)
tools = [retriever_tool]

In [ ]:
# Construct the Self Ask With Search Agent
from langchain.agents import AgentExecutor, create_self_ask_with_search_agent
agent = create_self_ask_with_search_agent(llm, tools, prompt)

In [ ]:
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools,
                               verbose=True, handle_parsing_errors=True)

docs = agent_executor.invoke(
    {"input": ""}
)

In [ ]:
for index, doc in enumerate(docs):
    #print ("{}\t{}.".format(index, doc.metadata['Title']))
    #print("\t{}".format(doc.metadata['Entry ID']))
    #print("\t{}".format(doc.metadata['Published']))
    print(doc)